In [1]:
#pip install py4j

In [2]:
import pyspark

In [3]:
pip install findspark

Note: you may need to restart the kernel to use updated packages.


In [4]:
import findspark
findspark.init('/Users/chenminjia/server/spark-3.0.1-bin-hadoop2.7')
import pyspark

In [5]:
from pyspark.sql import SparkSession

In [6]:
import os
exec(open(os.path.join(os.environ["SPARK_HOME"],'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.0.1
      /_/

Using Python version 3.8.3 (default, Jul  2 2020 11:26:31)
SparkSession available as 'spark'.


In [7]:
from pyspark.context import SparkContext
spark = SparkSession.builder.appName("spark test").getOrCreate()

In [8]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, StandardScaler, VectorAssembler, VectorSlicer, PCA
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.functions import vector_to_array
from pyspark.sql.functions import col, lit, udf
from pyspark.ml.clustering import KMeans
import re
from functools import reduce
import copy

from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.ml.linalg import SparseVector, DenseVector
from pyspark.sql.types import FloatType

from pyspark.sql import Row
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.types import FloatType
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import udf
from pyspark.sql import functions as F
from pyspark.sql.functions import explode, col, udf, mean as _mean, stddev as _stddev, log, log10
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.functions import lit
from pyspark.ml.feature import OneHotEncoder, StringIndexer, StandardScaler, VectorAssembler, VectorSlicer, PCA
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.functions import vector_to_array
from pyspark.ml.clustering import KMeans
import re
from functools import reduce

In [9]:
train_df = spark.read.csv('train_features.csv', header=True, inferSchema=True)
target_df = spark.read.csv('train_targets_scored.csv', header=True, inferSchema=True)
test_df = spark.read.csv('test_features.csv', header=True, inferSchema=True)

In [10]:
train_df.cache()
target_df.cache()
test_df.cache()

DataFrame[sig_id: string, cp_type: string, cp_time: int, cp_dose: string, g-0: double, g-1: double, g-2: double, g-3: double, g-4: double, g-5: double, g-6: double, g-7: double, g-8: double, g-9: double, g-10: double, g-11: double, g-12: double, g-13: double, g-14: double, g-15: double, g-16: double, g-17: double, g-18: double, g-19: double, g-20: double, g-21: double, g-22: double, g-23: double, g-24: double, g-25: double, g-26: double, g-27: double, g-28: double, g-29: double, g-30: double, g-31: double, g-32: double, g-33: double, g-34: double, g-35: double, g-36: double, g-37: double, g-38: double, g-39: double, g-40: double, g-41: double, g-42: double, g-43: double, g-44: double, g-45: double, g-46: double, g-47: double, g-48: double, g-49: double, g-50: double, g-51: double, g-52: double, g-53: double, g-54: double, g-55: double, g-56: double, g-57: double, g-58: double, g-59: double, g-60: double, g-61: double, g-62: double, g-63: double, g-64: double, g-65: double, g-66: double

In [11]:
train_df.printSchema()

root
 |-- sig_id: string (nullable = true)
 |-- cp_type: string (nullable = true)
 |-- cp_time: integer (nullable = true)
 |-- cp_dose: string (nullable = true)
 |-- g-0: double (nullable = true)
 |-- g-1: double (nullable = true)
 |-- g-2: double (nullable = true)
 |-- g-3: double (nullable = true)
 |-- g-4: double (nullable = true)
 |-- g-5: double (nullable = true)
 |-- g-6: double (nullable = true)
 |-- g-7: double (nullable = true)
 |-- g-8: double (nullable = true)
 |-- g-9: double (nullable = true)
 |-- g-10: double (nullable = true)
 |-- g-11: double (nullable = true)
 |-- g-12: double (nullable = true)
 |-- g-13: double (nullable = true)
 |-- g-14: double (nullable = true)
 |-- g-15: double (nullable = true)
 |-- g-16: double (nullable = true)
 |-- g-17: double (nullable = true)
 |-- g-18: double (nullable = true)
 |-- g-19: double (nullable = true)
 |-- g-20: double (nullable = true)
 |-- g-21: double (nullable = true)
 |-- g-22: double (nullable = true)
 |-- g-23: double (nu

In [12]:
target_df.printSchema()

root
 |-- sig_id: string (nullable = true)
 |-- 5-alpha_reductase_inhibitor: integer (nullable = true)
 |-- 11-beta-hsd1_inhibitor: integer (nullable = true)
 |-- acat_inhibitor: integer (nullable = true)
 |-- acetylcholine_receptor_agonist: integer (nullable = true)
 |-- acetylcholine_receptor_antagonist: integer (nullable = true)
 |-- acetylcholinesterase_inhibitor: integer (nullable = true)
 |-- adenosine_receptor_agonist: integer (nullable = true)
 |-- adenosine_receptor_antagonist: integer (nullable = true)
 |-- adenylyl_cyclase_activator: integer (nullable = true)
 |-- adrenergic_receptor_agonist: integer (nullable = true)
 |-- adrenergic_receptor_antagonist: integer (nullable = true)
 |-- akt_inhibitor: integer (nullable = true)
 |-- aldehyde_dehydrogenase_inhibitor: integer (nullable = true)
 |-- alk_inhibitor: integer (nullable = true)
 |-- ampk_activator: integer (nullable = true)
 |-- analgesic: integer (nullable = true)
 |-- androgen_receptor_agonist: integer (nullable = tr

In [13]:
def encode_cat_features(df, cat_features):

  indexed_cols = [''.join([col_name, '_indexed']) for col_name in cat_features]
  encoded_cols = [''.join([col_name, '_encoded']) for col_name in cat_features]
  string_indexers = [StringIndexer(inputCol=cat_features[i], outputCol=indexed_cols[i]) for i in range(len(cat_features))]
    
  encoder = OneHotEncoder(inputCols=indexed_cols, outputCols=encoded_cols)
  
  pipline = Pipeline(stages=string_indexers + [encoder])
  
  encoded_df = pipline.fit(df).transform(df)
  encoded_df = encoded_df.drop(*indexed_cols + cat_features)

  return encoded_df

def normalize_features(df, cols, normalizer, output_cols, if_drop=True):
  """
  """
  normalizer_lst = []
  vectorized_cols = []
  vector_assembers = []
  
  if isinstance(cols, list):
    cols = {'cols': cols}
  
  if isinstance(output_cols, str):
    output_cols = {'cols': output_cols}
  
  for k, v in cols.items():
    
    temp_normalizer = normalizer.copy()
    vectorized_col = ''.join([output_cols[k], '_v'])
    vector_assember = VectorAssembler(inputCols=v, outputCol=vectorized_col)
    
    temp_normalizer.setInputCol(vectorized_col)
    temp_normalizer.setOutputCol(output_cols[k])
    
    normalizer_lst.append(temp_normalizer)
    vectorized_cols.append(vectorized_col)
    vector_assembers.append(vector_assember)
  
  pipline = Pipeline(stages=vector_assembers + normalizer_lst)
  normalized_df = pipline.fit(df).transform(df).drop(*vectorized_cols)
  
  if if_drop:
    
    for k, v in cols.items():
      
      normalized_df = normalized_df.drop(v)
  
  return normalized_df

def add_pca_features(df, g_cols, c_cols, k=40):
  
  ## normalize g-col and c-col
  std_scaler = StandardScaler(withMean=True)
  
  input_cols = {
    'g_cols': g_cols, 
    'c_cols': c_cols}
  
  output_cols = {
    'g_cols': 'g_normalized', 
    'c_cols': 'c_normalized'}
  
  normalized_df = normalize_features(df, input_cols, std_scaler, output_cols, if_drop=False)
  
  ## perform PCA on g-cols and c-cols
  g_col_pca = PCA(k=k, inputCol='g_normalized', outputCol='g_col_pca')
  c_col_pca = PCA(k=k, inputCol='c_normalized', outputCol='c_col_pca')
  
  pipeline = Pipeline(stages=[g_col_pca, c_col_pca])
  pca_df = pipeline.fit(normalized_df).transform(normalized_df)
  
  return pca_df
  
def add_stats_features(df, g_cols, c_cols):
  
  @udf('double')
  def cols_sum(*lst):

    return sum(lst)

  @udf('double')
  def cols_mean(*lst):

    n = len(lst)
    s = sum(lst)

    return s / n

  @udf('double')
  def cols_var(*lst):

    n = len(lst)
    s = sum(lst) / n
    total = 0

    for x in lst:

      total += (x - s)**2 

    return total / n
  
  @udf('double')
  def cols_min(*lst):
    
    return min(lst)
  
  @udf('double')
  def cols_max(*lst):
    
    return max(lst)
  
  stats_dict = {
    'min_stats': cols_min,
    'max_stats': cols_max,
    'var_stats': cols_var,
    'mean_stats': cols_mean,
    'sum_stats': cols_sum
  }
  
  for name, func in stats_dict.items():
    
    df = df.withColumn(''.join(['g_cols_', name]), func(*[col(g_col) for g_col in g_cols]))
    df = df.withColumn(''.join(['c_cols_', name]), func(*[col(c_col) for c_col in c_cols]))
  
  return df

def add_kmeans_features(df, g_cols, c_cols, k=2, num_iter=10):
  
  kmeans_g = KMeans(k=k, featuresCol=g_cols, predictionCol='g_col_k_mean', seed=16)
  kmeans_c = KMeans(k=k, featuresCol=c_cols, predictionCol='c_col_k_mean', seed=16)
  
  kmeans_df = kmeans_g.fit(df).transform(df)
  kmeans_df = kmeans_c.fit(kmeans_df).transform(kmeans_df)
  
  return kmeans_df

def feature_engineering(df, num_cluster=2, num_comp=40, num_iter=10):
  
  ## get g-col and c-col
  g_cols = list(filter(lambda v: re.match('g-.+', v), df.columns))
  c_cols = list(filter(lambda v: re.match('c-.+', v), df.columns))
  
  ## PCA
  pca_df = add_pca_features(df, g_cols, c_cols, num_comp)

  ## stats features on g and c cols
  stats_df = add_stats_features(pca_df, g_cols, c_cols)
  
  ## add k-means features
  kmeans_df = add_kmeans_features(stats_df, g_cols='g_normalized', c_cols='c_normalized', k=num_cluster, num_iter=num_iter)
  
  return kmeans_df

In [14]:
## add indicator column to both train and test so we can combine them later
train_df = train_df.withColumn('is_test', lit(0))
test_df = test_df.withColumn('is_test', lit(1))

## Combine train and test df
full_df = train_df.union(test_df)

In [15]:
## encode features
target_cols = ['cp_type', 'cp_dose']
encoded_df = encode_cat_features(full_df, target_cols)

In [16]:
## feature engineering
fe_df = feature_engineering(encoded_df, num_comp=20, num_iter=5)

## select all the feature columns

pca_cols = list(filter(lambda v: re.match('.+_pca', v), fe_df.columns))
stats_cols = list(filter(lambda v: re.match('.+_stats', v), fe_df.columns))
k_means_cols = list(filter(lambda v: re.match('.+_k_mean', v), fe_df.columns))
cat_cols = list(filter(lambda v: re.match('.+_encoded', v), fe_df.columns)) + ['cp_time']

## stack them to a single feature vector
vector_assember_train = VectorAssembler(inputCols=pca_cols + stats_cols + k_means_cols + cat_cols, outputCol='all_features')
fe_df = vector_assember_train.transform(fe_df)

## normalize all the features
normalizer = StandardScaler(withMean=True)
cols = ['all_features']
output_cols = 'features'
fe_df = normalize_features(fe_df, cols, normalizer, output_cols, if_drop=False)

## split train, test df
fe_train = fe_df.filter(fe_df['is_test'] == 0)
final_test = fe_df.filter(fe_df['is_test'] == 1).select(['sig_id', 'features'])

## stack the training target
vector_assember_target = VectorAssembler(inputCols=[c for c in target_df.columns if c not in {'sig_id'}], outputCol='targets')
vectorized_target = vector_assember_target.transform(target_df).select(['sig_id', 'targets'])

## join training target with training features
final_train = fe_train.join(vectorized_target, ['sig_id']).select(['sig_id', 'features', 'targets'])

In [17]:
## train test split
(train, validation) = final_train.randomSplit([0.8, 0.2], 16)

train.cache()
validation.cache()

DataFrame[sig_id: string, features: vector, targets: vector]

In [18]:
train.show(5)

+------------+--------------------+--------------------+
|      sig_id|            features|             targets|
+------------+--------------------+--------------------+
|id_000644bb2|[0.44977285881488...|   (206,[101],[1.0])|
|id_000779bfc|[0.37428249793350...|         (206,[],[])|
|id_000a6266a|[0.09444594984942...|(206,[48,119,149]...|
|id_0015fd391|[-0.4887663847400...|         (206,[],[])|
|id_001626bd3|[0.41801707007827...|    (206,[54],[1.0])|
+------------+--------------------+--------------------+
only showing top 5 rows



In [116]:
class EnsembleBaseClfs:

    def __init__(self, base_clf, num_classes, feature_col, target_col):

        self.base_clf = base_clf
        self._trained_clfs = []
        self.num_classes = num_classes
        self.feature_col = feature_col
        self.target_col = target_col
        #self.joined_table = train
        #self.scores = train
        
    def fit(self, train):

        for c in range(num_classes):
          
            temp_clf = self.base_clf.copy()
            temp_train = train.withColumn('new_target', vector_to_array(self.target_col)[c])
            
            temp_clf.setLabelCol('new_target')
            temp_clf.setFeaturesCol(self.feature_col)
            
            temp_clf = temp_clf.fit(temp_train)
            self._trained_clfs.append(temp_clf)

        return self
      

    def predict(self, x_test):
        
        for c in range(num_classes):
          
            temp_clf = self.base_clf.copy()
            temp_train = train.withColumn('new_target', vector_to_array(self.target_col)[c])
            
            temp_clf.setLabelCol('new_target')
            temp_clf.setFeaturesCol(self.feature_col)
            
            temp_clf = temp_clf.fit(temp_train)
            
            if c ==0:     
                pred_df = temp_clf.transform(x_test).drop('prediction','rawPrediction')
                pred_df = pred_df.withColumn('probability', vector_to_array('probability')[1])
                pred_df = pred_df.withColumnRenamed('probability', f'probability{c+1}')
            
            else:
                pred_df = temp_clf.transform(pred_df).drop('prediction','rawPrediction')
                pred_df = pred_df.withColumn('probability', vector_to_array('probability')[1])
                pred_df = pred_df.withColumnRenamed('probability', f'probability{c+1}')
        
        return pred_df


In [125]:
def score(df, predictCol, targetsCol):
    import math
    @udf('double')
    def log_loss(y, y_hat):
        r = 0
        cut = 10**(-15)
        for t, p in zip(y, y_hat):
            p = max(min(p, 1-cut),cut)
            r += t * math.log(p) + (1 - t) * math.log(1 - p)
        return r/len(y)

    log_loss = df.select(log_loss(targetsCol, predictCol).alias('log_loss'))
    return log_loss.select((-_mean(col('log_loss'))).alias('score'))

In [148]:
base_clf = LogisticRegression(maxIter=2)
num_classes = 206
feature_col = 'features'
target_col = 'targets'
clf = EnsembleBaseClfs(base_clf, num_classes, feature_col, target_col).fit(train)
pred = clf.predict(validation)

In [118]:
#pred.show(5)

+------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      sig_id|            features|             targets|        probability1|        probability2|        probability3|
+------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|id_001762a82|[0.51473492401012...|   (206,[101],[1.0])|8.620248597466409E-4|0.002168214793513...|0.001678127286486...|
|id_0024bcd70|[0.25491709695681...|(206,[79,93],[1.0...|6.830897528340449E-4|0.001822198251817...|0.001581274258814...|
|id_0039a2ff9|[0.26516938751342...|   (206,[176],[1.0])|0.002541070924686901|0.002120803393003...|0.001313782771987...|
|id_00548fd5c|[0.14503941764000...|(206,[71,161],[1....|0.003567222651344937|6.262223669876775E-4|7.726478525492415E-4|
|id_006f6dbd6|[0.04248244160080...|   (206,[149],[1.0])|7.019100394331287E-4|5.985360257466544E-4|3.186852913393767...|
+------------+--------------------+-----

In [149]:
pred_vector_assember = VectorAssembler(inputCols = pred.drop('sig_id','features','targets').columns, outputCol = 'predictions')
result_df = pred_vector_assember.transform(pred).select(['sig_id', 'targets', 'predictions'])
result_df = result_df.withColumn('targets', vector_to_array('targets'))
result_df = result_df.withColumn('predictions', vector_to_array('predictions'))

In [150]:
#result_df.printSchema()
result_df.show(5)

+------------+--------------------+--------------------+
|      sig_id|             targets|         predictions|
+------------+--------------------+--------------------+
|id_001762a82|[0.0, 0.0, 0.0, 0...|[8.62024859746640...|
|id_0024bcd70|[0.0, 0.0, 0.0, 0...|[6.83089752834044...|
|id_0039a2ff9|[0.0, 0.0, 0.0, 0...|[0.00254107092468...|
|id_00548fd5c|[0.0, 0.0, 0.0, 0...|[0.00356722265134...|
|id_006f6dbd6|[0.0, 0.0, 0.0, 0...|[7.01910039433128...|
+------------+--------------------+--------------------+
only showing top 5 rows



In [151]:
df = result_df
predictCol = 'predictions'
targetsCol = 'targets'
score(df, predictCol, targetsCol).show()

+------------------+
|             score|
+------------------+
|0.0176287099892805|
+------------------+

